# Week 06 — Regression for Decisions (회귀/계수 해석/통제)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 다른 조건이 같다면, spend가 revenue에 어떤 영향이 있는가?
- discount가 margin에 미치는 영향은 채널별로 다른가?

## 2) Why this matters (Theory)
- 회귀는 '다른 조건 동일 시 영향'을 추정하는 도구
- log 변환은 비대칭 분포에서 해석/안정성에 도움
- 회귀는 인과를 보장하지 않음 → 실험/추가 검증 필요

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 OLS: log(revenue) ~ log(spend) + controls
✅ 통제 변수를 포함하여 '조건 동일 시' 효과를 추정합니다.

In [ ]:
m = df.dropna(subset=['marketing_spend']).copy()
m['log_spend'] = np.log1p(m['marketing_spend'])
m['log_revenue'] = np.log1p(m['revenue'].fillna(0))

model = smf.ols('log_revenue ~ log_spend + C(channel) + C(segment) + C(country) + discount_rate', data=m).fit()
print(model.summary())

In [ ]:
one_sentence_insight("log_spend 계수의 의미를 '대략 % 변화'로 해석해 한 문장")

### 5.2 Margin model: margin_rate ~ discount + controls
✅ 할인율이 마진율에 주는 영향을 정량화합니다.

In [ ]:
mm = m[m['revenue'].fillna(0) > 0].copy()
mm['margin_rate'] = mm['gross_margin'] / mm['revenue']
model2 = smf.ols('margin_rate ~ discount_rate + C(channel) + C(segment) + C(country)', data=mm).fit()
print(model2.summary())

In [ ]:
one_sentence_insight('discount가 마진을 얼마나 깎는지(리스크) 한 문장')

### 5.3 Interaction: discount effect by channel
✅ 채널별로 할인 민감도가 다른지 확인합니다.

In [ ]:
model3 = smf.ols('margin_rate ~ discount_rate * C(channel) + C(segment) + C(country)', data=mm).fit()
print(model3.summary())

In [ ]:
one_sentence_insight('어떤 채널에서 할인 리스크가 특히 큰지 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
